In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('max_rows',1000)

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subplots
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

line_color = ['#008abc','#96508e','#4fcb93','#f86e35','#ddaa18','#20beff']

from xgboost import XGBRegressor

from xgboost import plot_importance
from sklearn.inspection import partial_dependence, plot_partial_dependence

import datetime


In [ ]:
df_covid = pd.read_csv('../input/covid19panels/country_panel.csv',index_col=[0,1])

In [ ]:
def plot_graph(df,countries_to_view,column_to_plot,title):
    fig = go.Figure()

    count = 0
    for country in countries_to_view:
        fig.add_trace(go.Scatter(x=df.loc[(country,slice(None)),:].index.get_level_values(1), y=df.loc[(country,slice(None)),column_to_plot],connectgaps=True, name=df.loc[(country,slice(None)),:].index.get_level_values(0)[0],
                         #line=dict(color=line_color[count],width=4)))
                        line=dict(width=4)))

        count+=1
        
    fig.update_layout(title=title,
                       yaxis = dict(rangemode = 'tozero'))
    
    return fig

In [ ]:
countries_to_view = ['Australia','Germany','India' ,'Israel' ,'Sweden' ,'United Kingdom' ,'United States']

In [ ]:
df_covid['new_cases_per_million_7_day_average'] = df_covid['new_cases_per_million'].rolling(7).mean()
fig = plot_graph(df_covid,countries_to_view,'new_cases_per_million_7_day_average','New cases per million')
py.offline.iplot(fig)


In [ ]:
fig = plot_graph(df_covid,countries_to_view,'Delta_perc_sequences','Percentage of Sequences that are the Delta variant')
py.offline.iplot(fig)


In [ ]:
fig = plot_graph(df_covid,countries_to_view,'people_fully_vaccinated_per_hundred','Percentage of people fully vaccinated')
py.offline.iplot(fig)



In [ ]:
df_covid['workplaces_percent_change_from_baseline_7_day_average'] = df_covid['workplaces_percent_change_from_baseline'].rolling(7).mean()
fig = plot_graph(df_covid,countries_to_view,'workplaces_percent_change_from_baseline_7_day_average','Time spent at workplaces')
py.offline.iplot(fig)




In [ ]:
fig = plot_graph(df_covid,countries_to_view,'excess_mortality','Excess mortaility (measured as percentage increase above usual patterns)')
py.offline.iplot(fig)


In [ ]:
df_covid['perc_hospital_beds_for_covid'] = df_covid['hosp_patients_per_million']/(df_covid['hospital_beds_per_thousand']*1000)*100
fig = plot_graph(df_covid,countries_to_view,'perc_hospital_beds_for_covid','Percentage of hospital beds used by COVID patients')
py.offline.iplot(fig)


In [ ]:
df_covid['perc_hospital_beds_for_covid'] = df_covid['hosp_patients_per_million']/(df_covid['hospital_beds_per_thousand']*1000)*100
fig = plot_graph(df_covid,countries_to_view,'perc_hospital_beds_for_covid','Percentage of hospital beds used by COVID patients')
py.offline.iplot(fig)


In [ ]:
df_covid['mRNA_perc_of_total_vaccinations'] = df_covid['Moderna_perc_of_total_vaccinations'] + df_covid['Pfizer/BioNTech_perc_of_total_vaccinations']
fig = plot_graph(df_covid,countries_to_view,'mRNA_perc_of_total_vaccinations','Percentage of vaccines that are mRNA vaccines')
py.offline.iplot(fig)


In [ ]:
latest_date = (pd.to_datetime(df_covid.index.get_level_values(1).max())-datetime.timedelta(1)).strftime('%Y-%m-%d')
 
countries = df_covid.index.get_level_values(0).unique()
for country in countries:
    df_covid.loc[(country,slice(None)),'new_deaths_per_million_last_14_days'] = df_covid.loc[(country,slice(None)),'new_deaths_per_million'].rolling(14).sum()
    
df_death_rate = df_covid.loc[(slice(None),latest_date),['new_deaths_per_million_last_14_days','people_fully_vaccinated_per_hundred']]
df_death_rate = df_death_rate.droplevel('date', axis=0)

fig = px.scatter(df_death_rate, x="people_fully_vaccinated_per_hundred", y="new_deaths_per_million_last_14_days", text=df_death_rate.index, trendline="ols", width=1200, height=800,                 
                 labels={
                     "people_fully_vaccinated_per_hundred": "% of population that is full vaccinated",
                     "new_deaths_per_million_last_14_days": "New deaths per million",
                 },
                title="Relationship between death rate and vaccination rate"
                )

fig.update_traces(textposition='top center')

fig.show()


In [ ]:
latest_date = df_covid.index.get_level_values(1).max()
 
df_death_rate = df_covid.loc[(slice(None),latest_date),['new_deaths_per_million_last_14_days','people_fully_vaccinated_per_hundred']]
df_death_rate = df_death_rate.droplevel('date', axis=0)
df_death_rate.loc[:,'new_deaths_per_million_last_14_days_global_peak'] = df_covid.loc[(slice(None),'2021-01-20'),'new_deaths_per_million_last_14_days'].droplevel('date', axis=0)


df_death_rate_select_countries = df_death_rate[df_death_rate.index.get_level_values(0).isin(countries_to_view)]

fig = go.Figure(data=[
    go.Bar(name='COVID death rate in late Jan', x=df_death_rate_select_countries.index, y=df_death_rate_select_countries['new_deaths_per_million_last_14_days_global_peak']),
    go.Bar(name='COVID death rate now', x=df_death_rate_select_countries.index, y=df_death_rate_select_countries['new_deaths_per_million_last_14_days'])
])
# Change the bar mode
fig.update_layout(barmode='group',title="COVID death rate at global peak vs COVID death rate now")
fig.show()